# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Волков Станислав`

Группа: `ML-12`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [1]:
import random
import warnings
import geopandas as gpd
from shapely.geometry import Point
warnings.filterwarnings('ignore')

# Вытаскиваем данные границ Республики Афганистан
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
afganistan_polygon = world[world["name"] == "Afghanistan"]
afganistan_polygon = afganistan_polygon["geometry"].values[0]

#### Описание алгоритма
- Вытаскиваем границы полигона Республики Афганистан 
- Находим максимальные и минимальные точки щраницы полигона границ Республики Афганистан
- Запускаем цикл (пока точка не будет находится внтури полигона)
-- Генеририуем точки из полигона с границами (начинаем с минимальной и максимальной широты и долготы)
-- Уменьшаем границы полигона до граничных точек полигона на каждой итерации, в зависимости от того, с какой стороны от полигона была сгенерирована эта точка

In [2]:
from random import uniform

min_lat, max_lat = -90, 90
min_long, max_long = -180, 180
min_long_afg, min_lat_afg, max_long_afg, max_lat_afg = afganistan_polygon.bounds
print(afganistan_polygon.bounds)

random_point_list = []
while (True):
    latitude = uniform(min_lat, max_lat)
    longitude = uniform(min_long, max_long)
    print(longitude, latitude)
    random_point = Point(longitude, latitude)
    random_point_list.append(random_point)
    if random_point.within(afganistan_polygon):
        break
    else:
        if ((longitude >= min_long_afg) & (longitude <= max_long_afg)):
            min_long = min_long
            max_long = max_long
        elif longitude < min_long_afg:
            min_long = min_long_afg
        elif longitude > min_long_afg:
            max_long = max_long_afg
            
        if ((latitude >= min_lat_afg) & (latitude <= max_lat_afg)):
            min_lat = min_lat
            max_lat = max_lat
        elif latitude < min_lat_afg:
            min_lat = min_lat_afg
        elif latitude > min_lat_afg:
            max_lat = max_lat_afg            

(60.52842980331158, 29.31857249604431, 75.15802778514092, 38.486281643216415)
29.885640884318036 30.41268960695089
103.97160837704996 -20.798112148012734
70.97707124340671 58.60296283141668
75.05476977132057 35.89674603427876
69.29131660869848 35.03384000014643


In [3]:
import folium
m = folium.Map(
    location=[63.593095756151115, 35.72941851996457],
    zoom_start=2)

folium.GeoJson(
    afganistan_polygon
).add_to(m)

i = 0
for point in random_point_list:
    i += 1
    folium.Marker(
        location=[point.y, point.x],
        popup='iteration ' + str(i),
        icon=folium.Icon(icon='cloud')
    ).add_to(m)

In [4]:
m

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [5]:
%%time
## Для задания был сформирован список стран (наименования)
import pandas as pd
countries = pd.read_csv("oms_country_list.csv")
countries.head()

CPU times: user 2.7 ms, sys: 979 µs, total: 3.68 ms
Wall time: 3.47 ms


,country
0,Germany
1,United States
2,Russia
3,France
4,United Kingdom


In [6]:
%%time
## Для каждой страны находим areaId
from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()

# Вытаскиваем все areaID для всех стран
countries["areaId"] = countries["country"].apply(lambda x: str(nominatim.query(x).areaId()))

# Фильтруем areaID, которые оканчиваются на 0
countries = countries[countries.apply(lambda x: 1 if x["areaId"][-1] == "0" else 0, axis = 1) == 1]
countries.head()

CPU times: user 30.2 ms, sys: 16.6 ms, total: 46.8 ms
Wall time: 77.7 ms


,country,areaId
5,Brazil,3600059470
21,Australia,3600080500
23,United Republic of Tanzania,3600195270
34,Portugal,3600295480
44,Norway,3602978650


#### 1. Расстояние от точки до 5 ближайших * банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости

In [7]:
# https://wiki.openstreetmap.org/wiki/RU:%D0%9E%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D1%8B_%D0%BA%D0%B0%D1%80%D1%82%D1%8B
# Найдем кол-во жилой недвижимости для каждой анализируемой страны
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
overpass = Overpass()

residential = []
for index, value in countries.iterrows():
    query_residential = overpassQueryBuilder(area=value["areaId"], 
                                 elementType="node", 
                                 selector=['"building"="residential"'],
                                 out="count")
    result = overpass.query(query_residential, timeout=1000)
    residential.append(result.countElements())
    
# Для каждой страны подтягиваем кол-во жилой недвижимости   
countries["cnt_residential"] = residential

# Выводим страну с наибольшим кол-вом жилой недвижимости
countries[countries["cnt_residential"] == countries["cnt_residential"].max()]

,country,areaId,cnt_residential
63,Malawi,3600195290,8519


In [8]:
# 1 Найдем все банкоматы в Malawi
query_bank_in_malawi = overpassQueryBuilder(area=3600195290, 
                             elementType="node", 
                             selector=['"amenity"="atm"'],
                             out="body")
result = overpass.query(query_bank_in_malawi, timeout=1000)

malawi_bankomat_points = []
for res in result.nodes():
    malawi_bankomat_points.append(Point(res.lat(), res.lon()))
    
# 2 Найдем ближайшие 5 банкоматов
import rtree
idx = rtree.index.Rtree()
# Индексируем координаты
i = 1
for p in malawi_bankomat_points:
    idx.insert(i, (p.x, p.y), (p.x, p.y))
    i += 1
    
p = random_point_list[-1].y, random_point_list[-1].x
nearest_bankomats = list(idx.nearest(p, 5, objects='raw'))
nearest_bankomats

[(-9.5924951, 33.775929),
 (-9.9381732, 33.9266084),
 (-11.0227522, 33.8560264),
 (-11.6052415, 34.2975439),
 (-11.6086399, 34.2981484)]

In [9]:
# Найдем расстояние в метрах между 5 банкоматам и найденной рандомной точкой
import geopy.distance
last_random_point = (random_point_list[-1].y, random_point_list[-1].x)
for bankomats in nearest_bankomats:
    print(round(geopy.distance.distance(bankomats, last_random_point).km * 1000), "метров")

6193898 метров
6215654 метров
6318542 метров
6344493 метров
6344768 метров


#### 2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице

In [10]:
import overpass
api = overpass.API()

In [11]:
%%time
# Вытаскиваем все столицы
query = """
relation["admin_level"="2"]["type"="boundary"]["boundary"="administrative"];
node(r:"admin_centre");
out meta;
"""
response = api.get(query)

capitals = []
for cap in response["features"]:
    try:
        capitals.append({
            "country" : cap["properties"]["is_in"],
            "capital" : cap["properties"]["name:en"],
            "coordinates" : cap["geometry"]["coordinates"],
            "areaId" : str(nominatim.query(cap["properties"]["name:en"]).areaId())
        })
    except Exception as e:
        continue
        
capitals = pd.DataFrame(capitals)
capitals = capitals[capitals["areaId"] != "None"]
capitals.head(1)

CPU times: user 70.4 ms, sys: 19.2 ms, total: 89.7 ms
Wall time: 4.88 s


,country,capital,coordinates,areaId
0,"England, United Kingdom, UK, Great Britain, Eu...",London,"[-0.127647, 51.507322]",3600065606


In [13]:
# для каждой столицы посчитаем кол-во аптек
overpass = Overpass()
pharmacy = []
for index, value in capitals.iterrows():
    query_pharmacy = overpassQueryBuilder(area=value["areaId"], 
                             elementType="node", 
                             selector=['"amenity"="pharmacy"'],
                             out="count")
    result = overpass.query(query_pharmacy, timeout=1000)
    pharmacy.append(result.countElements())
    
capitals["cnt_farmacy"] = pharmacy
capitals[capitals["cnt_farmacy"] == capitals["cnt_farmacy"].max()].iloc[0,:]

country        England, United Kingdom, UK, Great Britain, Eu...
capital                                                   London
coordinates                               [-0.127647, 51.507322]
areaId                                                3600065606
cnt_farmacy                                                  789
Name: 0, dtype: object

In [14]:
# 1 Найдем все школы в Великобритании
query_school_in_gb = overpassQueryBuilder(area=3600065606, 
                             elementType="node", 
                             selector=['"amenity"="school"'],
                             out="body")
result = overpass.query(query_school_in_gb, timeout=1000)

gb_school_points = []
for res in result.nodes():
    gb_school_points.append(Point(res.lat(), res.lon()))
    
# 2 Найдем 5 ближайших школ
import rtree
idx = rtree.index.Rtree()

# Индексируем координаты
i = 1
for p in gb_school_points:
    idx.insert(i, (p.x, p.y), (p.x, p.y))
    i += 1
    
p = random_point_list[-1].y, random_point_list[-1].x
nearest_schools = list(idx.nearest(p, 5, objects='raw'))
nearest_schools

[(51.5563469, 0.2714625),
 (51.5640876, 0.2624164),
 (51.5692834, 0.24315009999999998),
 (51.6217494, 0.22598359999999998),
 (51.5849613, 0.2101285)]

In [15]:
# Найдем расстояние в метрах между 5 школами и найденной рандомной точкой
import geopy.distance
last_random_point = (random_point_list[-1].y, random_point_list[-1].x)
for school in nearest_schools:
    print(round(geopy.distance.distance(school, last_random_point).km * 1000), "метров")

5662794 метров
5663271 метров
5664492 метров
5664701 метров
5666461 метров


#### 3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части

In [16]:
# Найдем кол-во жилой недвижимости станция и автобусных остановок
railway, busstop = [], []
for index, value in countries.iterrows():
    query_railway = overpassQueryBuilder(area=value["areaId"], 
                                 elementType="node", 
                                 selector=['"railway"="halt"'],
                                 out="count")
    result = overpass.query(query_railway, timeout=1000)
    railway.append(result.countElements())
    
    query_busstop = overpassQueryBuilder(area=value["areaId"], 
                                 elementType="node", 
                                 selector=['"highway"="bus_stop"'], 
                                 out="count")
    result = overpass.query(query_busstop, timeout=1000)
    busstop.append(result.countElements())

countries["cnt_railway"] = railway
countries["cnt_busstop"] = busstop
countries["relat_railway_busstop"] = countries["cnt_railway"] / countries["cnt_busstop"]
countries[countries["relat_railway_busstop"] == countries["relat_railway_busstop"].max()]

,country,areaId,cnt_residential,cnt_railway,cnt_busstop,relat_railway_busstop
136,Cameroon,3600192830,15,17,69,0.246377


In [17]:
query_pharmacy = overpassQueryBuilder(area=3600192830, 
                             elementType="node", 
                             selector=['"amenity"="cinema"'],
                             out="count")
result = overpass.query(query_pharmacy, timeout=1000)

cameroon_cinema_points = []
for res in result.nodes():
    cameroon_cinema_points.append(Point(res.lat(), res.lon()))
    
# 2 Найдем 5 ближайших кинотеатров
import rtree
idx = rtree.index.Rtree()

# Индексируем координаты
i = 1
for p in cameroon_cinema_points:
    idx.insert(i, (p.x, p.y), (p.x, p.y))
    i += 1
    
p = random_point_list[-1].y, random_point_list[-1].x
nearest_cinema = list(idx.nearest(p, 5, objects='raw'))
nearest_cinema

TypeError: 'NoneType' object is not iterable

Нет объектов в Камеруне, из-за этого получаем ошибку в предыдущем чанке

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [18]:
import os
import ast
import time
import folium
import pyproj
import fiona as fn
from shapely import geometry
from openrouteservice import client
from folium import Map, Marker, LayerControl
from shapely.ops import cascaded_union, transform
from shapely.geometry import shape, Polygon, mapping, MultiPolygon, LineString, Point

def style_function(color): # To style data
    return lambda feature: dict(color=color)

def convert_to_point(string):
    coords = string[6:].replace("(", "").replace(")", "").split()
    coords = list(map(float, coords))
    return coords

In [19]:
# Загружаем расположение всех школ Нью-йорка
nw_schools = pd.read_csv("HW2_data/2019_-_2020_School_Point_Locations.csv")

# удалим цифры и знаки из адреса, для того, чтобы получить точные координаты улиц, где находятся школы
nw_schools["cor_address"] = nw_schools["Address"].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
nw_schools["cor_address"] = nw_schools["cor_address"].apply(lambda x: ''.join([i for i in x if i not in ["-", ".", "!", "_"]]))

# конвертируем координаты в список
nw_schools["the_geom"] = nw_schools["the_geom"].apply(lambda x: convert_to_point(x))
nw_schools = nw_schools[["the_geom", "cor_address", "City", "State"]]
print(nw_schools.shape)
nw_schools.head(1)

(1992, 4)


,the_geom,cor_address,City,State
0,"[-73.85554326311143, 40.70843092338607]",METROPOLITAN AVENUE,QUEENS,NY


In [20]:
# координаты точки A
res = nominatim.query("3849 5th Ave, New York, NY 10075")
a_x, a_y = res.toJSON()[0]["lat"], res.toJSON()[0]["lon"]

# координаты точки B
res = nominatim.query("39-21 Washington St, Brooklyn, NY 11201")
b_x, b_y = res.toJSON()[0]["lat"], res.toJSON()[0]["lon"]

coordinates = [[float(a_y), float(a_x)], [float(b_y), float(b_x)]]
coordinates

[[-73.9623161, 40.7789525], [-73.98938, 40.703279]]

In [21]:
# Параметры ввода и создаем карту
api_key = '5b3ce3597851110001cf62487f801ad2ebc0410b84000d1bd1b9b32f' #https://openrouteservice.org/sign-up
clnt = client.Client(key=api_key)

map_new_york = folium.Map(location=([40.703279, -73.95938]), zoom_start=12)
    
folium.Marker(location=[40.7789525, -73.9623161], popup='Central Park', 
icon=folium.Icon(icon='cloud')).add_to(map_new_york)

folium.Marker(location=[40.703279, -73.98938], popup='St, Brooklyn', 
icon=folium.Icon(icon='cloud')).add_to(map_new_york)
map_new_york

In [22]:
# Нарисуем простой короткий путь от A до B синим цветом
direction_params = {'coordinates': coordinates,
                    'profile': 'driving-car', 
                    'format_out': 'geojson',
                    'preference': 'shortest',
                    'geometry': 'true'}

regular_route = clnt.directions(**direction_params)

duration, distance = regular_route['features'][0]['properties']['summary'].values()

gj= folium.GeoJson(regular_route,
                   name='Regular Route',
                   style_function=style_function('blue')) \
          .add_to(map_new_york)
map_new_york

In [23]:
# Функции из примера по ссылке (https://openrouteservice.org/example-avoid-obstacles-while-routing/)
# Генерируем область вокруг координаты точки, где предположительно могут быть пешеходные переходы и проходит дорога
def CreateBufferPolygon(point_in, resolution=10, radius=10):    
    sr_wgs = pyproj.Proj(init='epsg:4326')
    sr_utm = pyproj.Proj(init='epsg:32632')
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in)
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution)
    
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point)) # Transform back to WGS84

    return poly_wgs

In [24]:
# Нарисуем все школы (пока 300, так как работать будет быстрей, но идея понятная)
sites_poly = []
for index, data in nw_schools.head(300).iterrows():
    site_coords = data["the_geom"]
    folium.features.Marker(list(reversed(site_coords)),
                          icon=folium.Icon(color='white',
                                               icon_color='#ff1a1a',
                                               prefix='fa'
                                              )
                          ).add_to(map_new_york)
    
    site_poly_coords = CreateBufferPolygon(site_coords, resolution=2, radius=10)
    sites_poly.append(site_poly_coords)
    
    site_poly_coords = [(y,x) for x,y in site_poly_coords]
    folium.vector_layers.Polygon(locations=site_poly_coords,
                                  color='red',
                                  fill_color='red',
                                  fill_opacity=0.2,
                                  weight=1).add_to(map_new_york)
    
map_new_york

In [27]:
# Создадим новую карту, в которой оставим школы, попадующие в полигон по пути из точки A в точку B
# Синие иконки - школы попадующие в полигон по пути из точки A в точку B
def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=0.5)


folium.features.GeoJson(data=regular_route,
                        name='Route with schools',
                        style_function=style_function("blue"),
                        overlay=True).add_to(map_new_york)

route_buffer = LineString(regular_route['features'][0]['geometry']['coordinates']).buffer(0.009)
folium.features.GeoJson(data=geometry.mapping(route_buffer),
                        name='Route Buffer',
                        style_function=style_function('#00FF51'),
                        overlay=True).add_to(map_new_york)

sites_buffer_poly = []
for site_poly in sites_poly:
    poly = Polygon(site_poly)
    if route_buffer.intersects(poly):
        folium.features.Marker(list(reversed(poly.centroid.coords[0]))).add_to(map_new_york)
        sites_buffer_poly.append(poly)

map_new_york

In [30]:
# Нарисуем оптимальный маршрут
direction_params['options'] = {'avoid_polygons': geometry.mapping(MultiPolygon(sites_buffer_poly))}
route_detour = clnt.directions(**direction_params)

folium.features.GeoJson(data=route_detour,
                        name='Route with construction sites',
                        style_function=style_function('green'),
                        overlay=True).add_to(map_new_york)

map_new_york.add_child(folium.map.LayerControl())
map_new_york